# PYTS/BOSSVS classification algorithm test and measured accuracies
# Tested on transformed data

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import math
import sys 
import shutil

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.metrics import f1_score

In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)       

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [5]:
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=5
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):


    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=6,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
                a=test_index[i]
                Q_ON.append(a)
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
                a=test_index[i]
                Q_ON.append(a)
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]



if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/7):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)


(1140, 3, 37)
Accuracy for ON class:  100.0 %
Accuracy for OFF class:  67.50103470217321 %
False Positive rate:  0.0 %
False Negative rate:  32.49896529782678 %
F1 score:  0.7832726086419057
[['4FGL J1443+2501.csv', 1], ['4FGL J1550+0528.csv', 1], ['4FGL J1558+5625.csv', 1], ['4FGL J1559+2319.csv', 2], ['4FGL J1600+8510.csv', 1], ['4FGL J1603+5009.csv', 1], ['4FGL J1604+5714.csv', 5], ['4FGL J1613+3411.csv', 1], ['4FGL J1615+2130.csv', 1], ['4FGL J1619+5536.csv', 2], ['4FGL J1621-1103.csv', 1], ['4FGL J1625+4134.csv', 1], ['4FGL J1630+5221.csv', 1], ['4FGL J1630+8234.csv', 3], ['4FGL J1635+3808.csv', 1], ['4FGL J1637+4717.csv', 1], ['4FGL J1638+5721.csv', 2], ['4FGL J1639+4129.csv', 1], ['4FGL J1640+3945.csv', 1], ['4FGL J1640+6850.csv', 1], ['4FGL J1642+3948.csv', 2], ['4FGL J1647+4950.csv', 1], ['4FGL J1649+5235.csv', 2], ['4FGL J1650+0831.csv', 1], ['4FGL J1651+7219.csv', 1], ['4FGL J1652+4024.csv', 1], ['4FGL J1656-2010.csv', 2], ['4FGL J1700+6830.csv', 2], ['4FGL J1701+3956.csv', 

In [11]:
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=5
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=1
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):


    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[0][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[0][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,binning)    
print(bigdata.shape)
for i in range(iteration):
    
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = BOSSVS(n_bins=6,anova=True,word_size=9,drop_sum=True)
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
                a=test_index[i]
                Q_ON.append(a)
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
                a=test_index[i]
                Q_ON.append(a)
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]



if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/7):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)

(1140, 37)
Accuracy for ON class:  14.754689754689753 %
Accuracy for OFF class:  87.80905901477242 %
False Positive rate:  85.24531024531024 %
False Negative rate:  12.19094098522759 %
F1 score:  0.887195231690303
[['4FGL J0056-2118.csv', 1], ['4FGL J0102+5824.csv', 1], ['4FGL J0115+2519.csv', 1], ['4FGL J0136+3906.csv', 1], ['4FGL J0205-1700.csv', 1], ['4FGL J0211+1051.csv', 1], ['4FGL J0217+7352.csv', 4], ['4FGL J0226-0553.csv', 2], ['4FGL J0231+1322.csv', 2], ['4FGL J0242+5216.csv', 2], ['4FGL J0319+1845.csv', 1], ['4FGL J0359+5057.csv', 2], ['4FGL J0426+6826.csv', 1], ['4FGL J0442-0017.csv', 1], ['4FGL J0444+3425.csv', 2], ['4FGL J0501-0158.csv', 1], ['4FGL J0505+0415.csv', 1], ['4FGL J0505+0459.csv', 2], ['4FGL J0521+2112.csv', 1], ['4FGL J0555+3947.csv', 2], ['4FGL J0608-1521.csv', 2], ['4FGL J0709-0255.csv', 2], ['4FGL J0723-0732.csv', 2], ['4FGL J0725+1425.csv', 1], ['4FGL J0730-1141.csv', 1], ['4FGL J0738+1742.csv', 1], ['4FGL J0739+0137.csv', 1], ['4FGL J0749+1058.csv', 1], [

In [14]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f) and filename in errOFF :
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
        
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)       

In [ ]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON

In [57]:

from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=200
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):

    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
                a=test_index[i]
                Q_ON.append(a)
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]
print(len(test_index))
print(len(np.unique(Q_ON)))

if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/8):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)

(366, 3, 37)
Accuracy for ON class:  100.0 %
Accuracy for OFF class:  93.19517316169927 %
False Positive rate:  0.0 %
False Negative rate:  6.80482683830072 %
F1 score:  0.9532897723752309
74
24
[['4FGLJ0047+2233.csv', 36], ['4FGLJ0203+7233.csv', 44], ['4FGLJ0205-1700.csv', 40], ['4FGLJ0209+4449.csv', 42], ['4FGLJ0209+7229.csv', 44], ['4FGLJ0211+1051.csv', 39], ['4FGLJ0212+2244.csv', 47], ['4FGLJ0214+5145.csv', 42], ['4FGLJ0215+0300.csv', 34], ['4FGLJ0216+2313.csv', 41], ['4FGLJ0216-1015.csv', 41], ['4FGLJ0217+0837.csv', 38], ['4FGLJ0217+7352.csv', 42], ['4FGLJ0217-0821.csv', 45], ['4FGLJ0219+2443.csv', 41], ['4FGLJ0219-1724.csv', 35]]


In [58]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f) and filename in errOFF :
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
        
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)

In [59]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON

In [60]:

from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=200
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore= []
FPOS=[]
FNEG=[]
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):

#         a=Q_ON2[j]

    dataframe=pd.read_csv(data_files_ALL[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
                a=test_index[i]
                Q_ON.append(a)
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))


gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]

if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>int(iteration/5):
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)

(124, 3, 37)


ZeroDivisionError: division by zero

In [ ]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f) and filename in errOFF :
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
        
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)

In [ ]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON

In [ ]:

from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier

iteration=200
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

binning=37

Labels = []
Q_ON= []

nbfeatures=3
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
a=binning
b=nbfeatures
c=len(data_files_ALL)

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

for j in range(len(data_files_ALL)):


    dataframe=pd.read_csv(data_files_ALL[j])
    dataframe.columns=['Iteration2','Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[2][j]=dataframe['Photon Index']

for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON): 
    
    
    Labels.append(int(1))
    
    
bigdata=bigdata.reshape(c,nbfeatures,binning)    
print(bigdata.shape)
for i in range(iteration):
    #3D shape of data, multivariate time series

    sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    X=bigdata
    y=Labels

    sss.get_n_splits(X, y)
    train_index, test_index = next(sss.split(X, y)) 

    x_train, x_test = X[train_index], X[test_index] 

    y_train=[]
    y_test=[]

    for i in range(len(train_index)):
        a=train_index[i]
        y_train.append(y[a])
    for i in range(len(test_index)):

        a=test_index[i]
        y_test.append(y[a])

    y_test2=y_test.copy()

    clf = MultivariateClassifier(BOSSVS(n_bins=5,anova=True,word_size=9,drop_sum=True))
    clf.fit(x_train, y_train)
    MultivariateClassifier(...)
    clf.predict(x_test)

    prediction=np.array(clf.predict(x_test))
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,prediction,average='weighted')
    fscore.append(f1)
    
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))



gg = np.unique(Q_ON)
Q_ON2=[]
Q_ON3=[]
errOFF=[]

if len(gg)>0:
    for i in range(len(Q_ON)):
        aa=Q_ON[i]
        Q_ON2.append(file_names_ALL[aa])
    gg = np.unique(Q_ON2)    
    for i in range(len(gg)):
        bb=gg[i]
        count = Q_ON2.count(bb)
        if count>1:
            Q_ON3.append([bb,count])
            errOFF.append(bb)
print(Q_ON3)